In [1]:
#from models.nalu import NALU
import torch.nn.functional as F
import torch
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from tqdm import tqdm_notebook as tqdm

from data_generator_helper import generate_synthetic_selection_dataset

from models.nalu import NALU
import torch
import torchvision.utils as vutils
import numpy as np
import torchvision.models as models
from torchvision import datasets
from tensorboardX import SummaryWriter
import datetime
import os

In [5]:
def reportLoss(loss):
    print(loss)
    
def train(model, optimizer, x_train, y_train, epochs, batch_size):
    for epoch in range(epochs):

        for batch in range(len(x_train) // batch_size):

            model.train()
            optimizer.zero_grad()

            x_batch_train = x_train[batch:(batch+batch_size),:]
            y_batch_train = y_train[batch:(batch+batch_size),:]

            out = model(x_batch_train)

            loss = F.mse_loss(out, y_batch_train)
            
            if loss != loss:
                print("nan detected")
            
            loss.backward()
            optimizer.step()

        if epoch % 10 == 0: 
            pass
            #reportLoss(loss.data)
            
    return test(model,x_train,y_train)
        
def test(model, x_test, y_test):
    
    model.eval()
    output_test = model(x_test)
    loss = F.mse_loss(output_test, y_test)
    
    return loss

In [7]:
test_per_range = 10
sample_size = 100
set_size = 1000

in_dim = sample_size
hidden_dim = 1
out_dim = 2
num_layers = 1

lr = 0.01
epochs = 500
batch_size = 10

values = np.linspace(1,1000,10)

train_loss = np.zeros((len(values),10))
test_loss = np.zeros((len(values),10))
print(values)
for j, value in tqdm(enumerate(values)):

    min_value = -value
    max_value = value
    
    train_acc = 0
    test_acc = 0
    for i in range(test_per_range):

        model = NALU(num_layers, in_dim, hidden_dim, out_dim)
        optimizer = torch.optim.RMSprop(model.parameters(),lr=lr)

        x_train, y_train, boundaries = generate_synthetic_selection_dataset(min_value, max_value,
                                                                    sample_size, set_size, boundaries = None)

        x_test, y_test, _ = generate_synthetic_selection_dataset(min_value, max_value,
                                                                    sample_size, set_size, boundaries = boundaries)

        train_loss[j, i] = train(model, optimizer, x_train, y_train, epochs, batch_size)

        test_loss[j, i]  = test(model, x_test, y_test)
    


[   1.  112.  223.  334.  445.  556.  667.  778.  889. 1000.]


NameError: name 'train_lost' is not defined

In [11]:
print(np.round(train_loss,0))
print()
print()
print(np.round(test_loss,0))

[[5.00000000e+00 4.00000000e+00 4.00000000e+00 3.00000000e+00
  3.00000000e+00 2.20000000e+01 5.00000000e+00 3.00000000e+00
  4.00000000e+00 4.00000000e+00]
 [1.41491000e+05 1.21995000e+05 4.08200000e+04 2.20510000e+05
  2.45469808e+08 1.44750000e+04 1.61412000e+05 4.47281000e+05
  8.95774000e+05 1.75991665e+10]
 [1.23117700e+06 3.12619451e+21 5.60433000e+05 2.03818000e+05
  5.24849000e+05 1.12442611e+24 7.64491000e+05 4.07334000e+05
  3.05926000e+05 2.26341370e+23]
 [4.85907000e+05 2.73656000e+05 1.78928345e+24 8.39173000e+05
  8.23115395e+25 1.57607270e+23 9.69050080e+07 2.33036200e+06
  3.14791391e+22 1.03217577e+23]
 [2.38814229e+28 2.36391890e+17 2.83447789e+29 4.11776987e+27
  1.53363443e+09 1.10385800e+06 1.20416481e+26 2.45311200e+06
  4.51077620e+23 4.27761954e+26]
 [1.21720567e+24 1.03545300e+06 1.43134383e+24 4.67993393e+23
  6.52361126e+25 5.17992859e+30 1.43072982e+25 1.50654500e+06
  1.58664601e+21 9.83588418e+25]
 [3.76683114e+29 1.10752811e+28 2.26554021e+28 4.07370000e

In [28]:
np.savetxt("all_train_loss_exp1.csv", np.mean(train_loss,axis=1), delimiter=',', fmt='%2.2f')
np.savetxt("all_test_loss_exp1.csv", np.mean(test_loss,axis=1), delimiter=',', fmt='%2.2f')

(array([0]), array([3]))